In [1]:
# pip
# !apt install git-lfs

!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install jiwer
!pip install numpy==1.23.4

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# import
import os
import torch

### GPU

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Enable CUDA error logging
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'

if torch.cuda.is_available():
    os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3,4,5,6,7'

### Load Dataset

In [4]:
from datasets import load_from_disk

# import local dataset
train_dataset = load_from_disk("./data/datasets/train")
eval_dataset = load_from_disk("./data/datasets/eval")

/home/haram/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import random
import pandas as pd
from IPython.display import display, HTML

# 무작위로 샘플 보여주는 함수 정의
def show_random_elements(dataset, num_examples=7):
    assert num_examples <= len(dataset)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [6]:
# 한 번 확인하기
# print("===========train=================")
print(len(train_dataset))
# show_random_elements(train_dataset)
# print("===========eval=================")
# show_random_elements(eval_dataset)
print(len(eval_dataset))

92664
11350


In [7]:
# 특수문자 제거
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch

In [8]:
# 특수문자 제거된 것 적용
train_dataset = train_dataset.map(remove_special_characters)
eval_dataset = eval_dataset.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7f373c82e710> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
92664ex [00:05, 17611.81ex/s]
11350ex [00:00, 19728.16ex/s]


### Tokenizer 만들기

In [9]:
import psutil

def extract_all_chars(batch):
    # 메모리 관련 로직 처리
    memory = psutil.virtual_memory()
    threshold = 80

    if memory.percent >= threshold:
        print("메모리 위험 :",memory.percent)
        return None
        
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [10]:
vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_eval = eval_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=eval_dataset.column_names)

100%|██████████| 1/1 [00:00<00:00, 13.67ba/s]


In [11]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_eval["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

In [12]:
vocab_dict

{'k': 0,
 'q': 1,
 ' ': 2,
 'p': 3,
 'z': 4,
 'j': 5,
 'x': 6,
 'c': 7,
 'l': 8,
 't': 9,
 'u': 10,
 'h': 11,
 'f': 12,
 'a': 13,
 'v': 14,
 "'": 15,
 'w': 16,
 'n': 17,
 'b': 18,
 'y': 19,
 'o': 20,
 'm': 21,
 's': 22,
 'e': 23,
 'g': 24,
 'i': 25,
 'd': 26,
 'r': 27}

In [13]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [14]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [15]:
# vocab json 저장
import json

model_path = "./model/wav2vec_second"
vocab_json = model_path+"/vocab.json"


with open(vocab_json, 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [16]:
# tokenizer 생성
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(vocab_json, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [17]:
tokenizer.save_pretrained(model_path)

('./model/wav2vec_second/tokenizer_config.json',
 './model/wav2vec_second/special_tokens_map.json',
 './model/wav2vec_second/vocab.json',
 './model/wav2vec_second/added_tokens.json')

### model에 대한 특성 정의

In [18]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [19]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [20]:
### 전처리 데이터 확인

In [21]:
train_dataset[0]["path"]

'./data/x_train/7294/86026/7294-86026-0025.flac'

In [22]:
eval_dataset[0]["path"]

'./data/x_eval/7294/86026/7294-86026-0001.flac'

In [23]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_dataset))

print(train_dataset[rand_int]["transcription"])
ipd.Audio(data=np.asarray(train_dataset[rand_int]["audio"]["array"]), autoplay=False, rate=16000)

print("Target text:", train_dataset[rand_int]["transcription"])
print("Input array shape:", np.asarray(train_dataset[rand_int]["audio"]["array"]).shape)
print("Sampling rate:", train_dataset[rand_int]["audio"]["sampling_rate"])

when he had finished his letters stepan arkadyevitch moved the office papers close to him rapidly looked through two pieces of business made a few notes with a big pencil and pushing away the papers turned to his coffee as he sipped his coffee he opened a still damp morning paper 
Target text: when he had finished his letters stepan arkadyevitch moved the office papers close to him rapidly looked through two pieces of business made a few notes with a big pencil and pushing away the papers turned to his coffee as he sipped his coffee he opened a still damp morning paper 
Input array shape: (255040,)
Sampling rate: 16000


In [24]:
rand_int = random.randint(0, len(eval_dataset))

print(eval_dataset[rand_int]["transcription"])
ipd.Audio(data=np.asarray(eval_dataset[rand_int]["audio"]["array"]), autoplay=False, rate=16000)

print("Target text:", eval_dataset[rand_int]["transcription"])
print("Input array shape:", np.asarray(eval_dataset[rand_int]["audio"]["array"]).shape)
print("Sampling rate:", eval_dataset[rand_int]["audio"]["sampling_rate"])

that youth has many generous impulses which do not last and that among them are some which being gratified become only the more fleeting above all i think said the lady fixing her eyes on her son's face that if an enthusiastic 
Target text: that youth has many generous impulses which do not last and that among them are some which being gratified become only the more fleeting above all i think said the lady fixing her eyes on her son's face that if an enthusiastic 
Input array shape: (249040,)
Sampling rate: 16000


In [25]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [26]:
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names,num_proc=8)
print(train_dataset)

 #0: 0ex [00:00, ?ex/s]
 #1: 0ex [00:00, ?ex/s]

 #2: 0ex [00:00, ?ex/s]


 #3: 0ex [00:00, ?ex/s]



 #4: 0ex [00:00, ?ex/s]




 #5: 0ex [00:00, ?ex/s]






 #7: 0ex [00:00, ?ex/s]





 #6: 0ex [00:00, ?ex/s]2024-04-27 03:46:00.517140: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-27 03:46:00.555008: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 03:46:00.555046: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 03:46:00.555067: E tensorflow/compiler/xla/stream_executor/c

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 92664
})


/home/haram/.local/lib/python3.10/site-packages/datasets/table.py:690: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [27]:
eval_dataset = eval_dataset.map(prepare_dataset, remove_columns=eval_dataset.column_names,num_proc=8)
print(eval_dataset)

 #0: 0ex [00:00, ?ex/s]
 #1: 0ex [00:00, ?ex/s]

 #2: 0ex [00:00, ?ex/s]


 #3: 0ex [00:00, ?ex/s]



 #4: 0ex [00:00, ?ex/s]




 #5: 0ex [00:00, ?ex/s]





 #6: 0ex [00:00, ?ex/s]






 #7: 0ex [00:00, ?ex/s]2024-04-27 03:48:27.733497: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-27 03:48:27.773554: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-27 03:48:27.776545: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 11350
})


In [28]:
# max_input_length_in_sec =.0
# train_dataset = train_dataset.filter(lambda x: x > max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

### trainer

In [29]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [30]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [31]:
from datasets import load_metric

wer_metric = load_metric("wer")

In [32]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [33]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
).to(device)

/home/haram/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/home/haram/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_q.bias', 'project_hid.weight']
- This IS expected i

In [34]:
model.freeze_feature_encoder()

In [35]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir=repo_name,
  output_dir=model_path,
  group_by_length=True,
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=300,
  eval_steps=300,
  logging_steps=300,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

2024-04-27 03:48:58.730198: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-27 03:48:58.775671: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 03:48:58.775720: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 03:48:58.775752: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-27 03:48:58.784644: I tensorflow/core/platform/cpu_feature_g

In [36]:
print(len(train_dataset))
print(len(eval_dataset))

92664
11350


In [37]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.


In [ ]:
trainer.save_model(model_path)